---
title: "Message bank Tutorial"
author: "Andrew Han"
date: "2024-02-21"
categories: [week 5, homework]
---

# Introduction

In this blog post, I will be giving a short tutorial on how to make a simple message bank using Python's Flask library, as well as incorporating HTML and CSS files to develop a webpage

First, we create the necessary files. To start we will create our app.py file, which will be a Python script file that will be running the Flask library tasks. Please see below.

In [ ]:
from flask import Flask, render_template, request
from flask import redirect, url_for, abort
from flask import g
import sqlite3

app = Flask(__name__)

# Going through app.py file

Along with importing the necessary libraries, we must define a few functions and databases for our message bank. Please see the comments above each function to understand what each function is doing in the context of the webpage.

In [ ]:
# Create a sqlite3 database to handle all of the submitted messages
DATABASE = 'messages_db.sqlite'

# This function checks for the existence of a message_db in the global 'g' object. It then closes the database connection if it sees its existence.
@app.teardown_appcontext
def close_db(error):
    if hasattr(g, 'message_db'):
        g.message_db.close()

@app.route('/') # @app.route essentially tells the code what the end of the url of this particular page would be. 
# Because this is the base 'Home' page, we use a simple '/' to conclude the url.
def base():
    #Uses Flask's render_template function to render the base.html file
    return render_template('base.html')


# Connection between app.py and each page's html file.

After defining a base() function, we must accompany it with an html file to tell the website what to display on this page. See below for the code for base.html.

In [ ]:
<!doctype html>
<link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
<title>{% block title %}{% endblock %} - PIC16B Website</title>
<nav>
    <h1>PIC16B Message Bank!</h1>
    <ul>
        <li><a href="{{ url_for('base') }}">Main page</a></li>
        <li><a href="{{ url_for('submit_message')}}">Submit Message</a></li>
        <li><a href="{{ url_for('render_view_template') }}">View Messages</a></li>
    </ul>
<nav>
<section class="content">
    <header>
    {% block header %}{% endblock %}
    </header>
    {% block content %}{% endblock %}
</section>

# base.html

What this code is doing is both serving as a base for additional html files, as well as formatting the base page. It lays out a page title, as well as navigation functionality to visit other pages. See below for the app.py function that renders the message submission page, as well as the accompanying submit.html file.

In [ ]:
# Like before, we use @app.route. 
#However, now we must define the methods to ensure that user submission is possible by adding methods=['GET, POST']

@app.route('/submit/', methods=['GET', 'POST'])
def submit_message():
    #different methods for diff request methods
    if request.method == 'GET':
        #if 'GET' just render the template
        return render_template('submit.html')
    
    else:
        try:
            handle, message = insert_message(request)
            #render the template with a thank you note
            return render_template('submit.html', thank_you = True, message = message, handle = handle)
        except:
            return render_template('submit.html', error=True)

In [ ]:
{% extends "base.html" %} 

{% block header %}
  <h1>{% block title %}Submit Message{% endblock %}</h1>
{% endblock %}

{% block content %}
    <form action="/submit" method="post">
        <label for="message">Message:</label><br>
        <input type="text" id="message" name="message"><br>
        <label for="name">Name:</label><br>
        <input type="text" id="name" name="name"><br>
        <input type="submit" value="Submit">
    </form>
{% endblock %}

# Message Submission Page

This page is run by the function submit_message(), which essentially receives a message from the user and renders the submit.html page. The submit.html file "extends" the base.html file, which essentially means that many components, like the style.css, is automatically transferred from the base.html file to submit.html. See below for a screenshot of this page.

#%%[markdown]

![MessageSubmission](/Users/andrewhan/Desktop/MessageSubmissionSS.png)


# Database Management

See below for the code for inserting a message into the SQL message database.

In [ ]:
@app.route('/getmsg/', methods=['GET'])
def get_message_db():

    #try to retrieve database connection from global g object
    try:
        db = g.message_db
    except AttributeError:
        db = g.message_db = sqlite3.connect(DATABASE)
        db.execute('''
            CREATE TABLE IF NOT EXISTS messages (
                id INTEGER PRIMARY KEY,
                handle TEXT,
                message TEXT
            )
        ''')
    return db

@app.route('/insert_msg/', methods=['POST'])
def insert_message(request):
    # get message and handle from request
    try:
        message = request.form['message']
        handle = request.form['name']

        db = get_message_db()
        cursor = db.cursor()
        cursor.execute("INSERT INTO messages (handle, message) VALUES (?, ?)", (handle, message))
        db.commit()

        return redirect('/submit/')
    except Exception as e:
        return render_template('submit.html', error=True)

# insert_message(request)

What this function does is connect to the message database and inserts the user-inputted message into the database so that it can be displayed in the message viewing page. 

See below for how the message viewing page is run.

# Random Messages Page

In [ ]:
@app.route('/random_messages/<int:n>')
def random_message(n):
    db = get_message_db()
    cursor = db.cursor()
    cursor.execute("SELECT * FROM messages ORDER BY RANDOM() LIMIT ?", (n,))
    messages = cursor.fetchall()
    return render_template('view.html', messages=messages)


@app.route('/random_messages/')
def render_view_template():
    messages = random_message(5)

    return render_template('view.html', messages = messages)

These functions connect to the message database to return and render a list of previously inputted, randomly selected messages to the user. See below for the view.html file.

In [ ]:
{% extends "base.html" %}

{% block content %}
    <h2>Random Messages</h2>
    <ul>
        {% for message in messages %}
            <li>
                <strong>{{ message.handle }}</strong> - {{ message.message }}
            </li>
        {% endfor %}
    </ul>
{% endblock %}

# view.html

This code also "extends" base.html. This html code is much simpler as it simply lists the previous messages, and doesn't need to receive any information from the website user. See an example below.

#%%[markdown]
Random
![MessageSubmission](/Users/andrewhan/Desktop/RandomMessagesSS.png)

# style.css

We have now seen how the website functions on the back-end with the database management and user inputs. However, this must all be presentable. As I mentioned earlier, the base.html file is accompanies by a style.css file. See below for the css code.

In [ ]:
/* Import a custom font from Google Fonts */
@import url('https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap');

/* Apply the custom font to the entire body */
body {
    font-family: 'Roboto', sans-serif;
    background-color: #f0f0f0; /* Light gray background */
    color: #333; /* Dark gray text */
}

/* Style the navigation bar */
nav {
    background-color: #333; /* Dark background color */
    color: white; /* White text */
    padding: 10px;
}

/* Style navigation links */
nav a {
    color: white; /* White text */
    text-decoration: none; /* Remove underline */
    margin-right: 10px; /* Add some margin between links */
}

/* Style the header section */
header {
    background-color: #007bff; /* Blue background color */
    color: white; /* White text */
    padding: 20px;
    text-align: center; /* Center align text */
}

/* Add some margin around the content */
.content {
    margin: 20px;
}

/* Style forms */
form {
    margin-bottom: 20px; /* Add margin below forms */
}

/* Style text inputs and buttons */
input[type="text"], button {
    padding: 10px;
    margin-right: 10px;
}

/* Style buttons */
button {
    background-color: #007bff; /* Blue background color */
    color: white; /* White text */
    border: none; /* Remove border */
    cursor: pointer; /* Change cursor to pointer */
}

/* Change button color on hover */
button:hover {
    background-color: #0056b3; /* Darker blue on hover */
}

/* Style unordered lists */
ul {
    list-style-type: none; /* Remove bullet points */
    padding: 0; /* Remove default padding */
}

/* Add some margin below list items */
li {
    margin-bottom: 10px;
}

This code goes through many different customizations, such as font type, font size and color, page color, etc.

# Conclusion

As we can see, there are many different things to account for in web development. Hopefully, this blog post was informational and helpful in developing your next webpage. Thank you!

- Andrew Han